In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from sklearn.cluster import DBSCAN
from google.colab.patches import cv2_imshow



In [ ]:
def leer_imagen_pgm(ruta):
    """Lee imágenes PGM (formatos P2/P5)"""
    with open(ruta, 'rb') as f:
        header = f.readline().decode('utf-8').strip()
        while True:
            line = f.readline().decode('utf-8').strip()
            if not line.startswith('#'):
                break
        width, height = map(int, line.split())
        max_val = int(f.readline().decode('utf-8').strip())

        if header == 'P5':
            image_data = np.frombuffer(f.read(), dtype=np.uint8)
        else:  # P2
            image_data = np.array([int(x) for x in f.read().decode('utf-8').split()], dtype=np.uint8)

        return image_data.reshape((height, width))

def extraer_roi_mamografia(imagen):
    """Extrae la región de interés (mama) excluyendo texto/marcas"""
    thresh = cv2.adaptiveThreshold(imagen, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY_INV, 201, 2)
    kernel = np.ones((25,25), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return (0, 0, imagen.shape[1], imagen.shape[0])

    main_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(main_contour)
    margin_x, margin_y = int(w*0.1), int(h*0.1)

    return (
        max(x - margin_x, 0),
        max(y - margin_y, 0),
        min(w + 2*margin_x, imagen.shape[1] - x),
        min(h + 2*margin_y, imagen.shape[0] - y)
    )

def detectar_microcalcificaciones(roi):
    """Detección con parámetros optimizados para mamografías"""
    circles = cv2.HoughCircles(
        roi,
        cv2.HOUGH_GRADIENT,
        dp=1.3,
        minDist=12,
        param1=65,
        param2=16,
        minRadius=1,
        maxRadius=12
    )
    return np.round(circles[0, :]).astype("int") if circles is not None else []

def filtrar_agrupaciones(circles, max_dist=15, min_samples=3):
    """Identifica agrupaciones sospechosas usando DBSCAN"""
    if len(circles) == 0:
        return [], []

    coords = np.array([[x, y] for (x, y, r) in circles])
    clustering = DBSCAN(eps=max_dist, min_samples=min_samples).fit(coords)
    labels = clustering.labels_

    return (
        [circles[i] for i in range(len(circles)) if labels[i] != -1],
        [circles[i] for i in range(len(circles)) if labels[i] == -1]
    )

def generar_mapa_densidad(circles, shape, sigma=15):
    """Genera mapa de densidad térmica"""
    mapa = np.zeros(shape[:2])
    for (x, y, _) in circles:
        if 0 <= y < shape[0] and 0 <= x < shape[1]:
            mapa[int(y), int(x)] += 1
    return gaussian_filter(mapa, sigma=sigma)


In [ ]:
def procesar_y_guardar(imagen_path, output_dir):
    """Procesa una imagen y guarda todos los resultados"""
    try:
        # Cargar y preprocesar
        imagen = leer_imagen_pgm(imagen_path)
        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(12,12))
        img_procesada = clahe.apply(imagen)

        # Extraer ROI y detectar
        x, y, w, h = extraer_roi_mamografia(img_procesada)
        roi = img_procesada[y:y+h, x:x+w]
        circles = detectar_microcalcificaciones(roi)

        # Ajustar coordenadas al ROI
        circles[:, 0] += x
        circles[:, 1] += y

        # Filtrar agrupaciones
        agrupaciones, aislados = filtrar_agrupaciones(circles)

        # Generar outputs
        output_img = cv2.cvtColor(imagen, cv2.COLOR_GRAY2BGR)
        for (x_c, y_c, r) in aislados:
            cv2.circle(output_img, (x_c, y_c), r, (0, 0, 255), 1)
        for (x_c, y_c, r) in agrupaciones:
            cv2.circle(output_img, (x_c, y_c), r, (0, 255, 0), 2)

        mapa_densidad = generar_mapa_densidad(circles, imagen.shape)

        # Clasificación BI-RADS
        birads = "4/5" if len(agrupaciones) >=5 else "3" if len(agrupaciones) >=2 else "2"

        # Guardar resultados
        filename = os.path.splitext(os.path.basename(imagen_path))[0]
        os.makedirs(output_dir, exist_ok=True)

        cv2.imwrite(f"{output_dir}/{filename}_procesada.png", output_img)
        plt.imsave(f"{output_dir}/{filename}_densidad.png", mapa_densidad, cmap='hot')

        with open(f"{output_dir}/{filename}_reporte.txt", 'w') as f:
            f.write(f"Reporte para {filename}.pgm\n")
            f.write("========================\n")
            f.write(f"Total microcalcificaciones: {len(circles)}\n")
            f.write(f"Agrupaciones sospechosas: {len(agrupaciones)}\n")
            f.write(f"Clasificación BI-RADS: {birads}\n")

        return True

    except Exception as e:
        print(f"Error procesando {imagen_path}: {str(e)}")
        return False


In [ ]:
# Configuración de rutas
input_folder = "/content/drive/MyDrive/MIAS"
output_folder = "/content/drive/MyDrive/MIAS_procesadas"

# Procesar todas las imágenes
for filename in sorted(os.listdir(input_folder)):
    if filename.lower().endswith('.pgm'):
        print(f"\nProcesando: {filename}")
        imagen_path = os.path.join(input_folder, filename)
        success = procesar_y_guardar(imagen_path, output_folder)

        if success:
            print(f"✓ Resultados guardados en {output_folder}")
        else:
            print(f"✗ Error al procesar {filename}")

print("\nProcesamiento completado. Revise los resultados en:", output_folder)


Procesando: mdb001.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb002.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb003.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb004.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb005.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb006.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb007.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb008.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb009.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb010.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb011.pgm
✓ Resultados guardados en /content/drive/MyDrive/MIAS_procesadas

Procesando: mdb012.p

In [ ]:
def generar_reporte_consolidado(output_folder):
    reporte_path = os.path.join(output_folder, "reporte_consolidado.txt")

    with open(reporte_path, 'w') as reporte:
        # Encabezado del reporte
        reporte.write("REPORTE CONSOLIDADO DE MICROCALCIFICACIONES\n")
        reporte.write("==========================================\n\n")
        reporte.write("{:<30} {:<15} {:<15} {:<10}\n".format(
            "Archivo", "Total Microcalc", "Agrupaciones", "BI-RADS"))
        reporte.write("-"*70 + "\n")

        # Leer reportes individuales
        for filename in sorted(os.listdir(output_folder)):
            if filename.endswith("_reporte.txt"):
                archivo_original = filename.replace("_reporte.txt", ".pgm")
                with open(os.path.join(output_folder, filename), 'r') as f:
                    lineas = f.readlines()
                    total = lineas[2].split(":")[1].strip()
                    agrup = lineas[3].split(":")[1].strip()
                    birads = lineas[4].split(":")[1].strip()

                    reporte.write("{:<30} {:<15} {:<15} {:<10}\n".format(
                        archivo_original, total, agrup, birads))

        # Estadísticas finales
        reporte.write("\nRESUMEN ESTADÍSTICO:\n")
        reporte.write("-------------------\n")

            len([f for f in os.listdir(output_folder) if f.endswith("_reporte.txt")])))

    print(f"\nReporte consolidado generado en: {reporte_path}")
    return reporte_path

# Generar el reporte final
reporte_final = generar_reporte_consolidado(output_folder)


Reporte consolidado generado en: /content/drive/MyDrive/MIAS_procesadas/reporte_consolidado.txt


In [21]:
from google.colab import files
import cv2
import numpy as np


uploaded = files.upload()
filename = list(uploaded.keys())[0]


imagen = cv2.imread(filename, cv2.IMREAD_UNCHANGED)


nuevo_nombre = filename.replace('.pgm', '.png')
cv2.imwrite(nuevo_nombre, imagen)

print(f"Conversión completa. Imagen guardada como {nuevo_nombre}")

Saving mdb001.pgm to mdb001.pgm
Conversión completa. Imagen guardada como mdb001.png


In [22]:
from google.colab import files

files.download(nuevo_nombre)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>